In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train=pd.read_csv('X_train.csv')
y_train=pd.read_csv('y_train.csv')
X_test=pd.read_csv('X_test.csv')
y_test=pd.read_csv('y_test.csv').values
car_id=y_test[:,0].reshape(-1,1)
y_test=y_test[:,1].reshape(-1,1)
y_test

array([[38000],
       [23495],
       [59999],
       ...,
       [21995],
       [18700],
       [ 2999]], dtype=int64)

In [3]:
X_train.head()

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


In [4]:
X_train.drop(columns=['model','carID'],axis=1,inplace=True)
X_test.drop(columns=['model','carID'],axis=1,inplace=True)

In [5]:
X_train.isnull().sum()

brand           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [6]:
X_train['fuelType'].value_counts()

Diesel      2989
Petrol      1754
Hybrid       189
Other         26
Electric       2
Name: fuelType, dtype: int64

In [7]:
X_train['fuelType']=X_train['fuelType'].map({"Diesel":0,"Petrol":1,"Hybrid":2,"Other":3,"Electric":4})
X_test['fuelType']=X_test['fuelType'].map({"Diesel":0,"Petrol":1,"Hybrid":2,"Other":3,"Electric":4})
X_train.head()

,brand,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,hyundi,2019,Semi-Auto,4223,0,145.0,39.8,2.2
1,vauxhall,2015,Manual,47870,0,125.0,60.1,2.0
2,audi,2019,Automatic,5151,1,145.0,29.1,2.9
3,vw,2016,Automatic,20423,0,30.0,57.6,2.0
4,skoda,2020,Semi-Auto,3569,1,145.0,47.1,1.0


In [8]:
X_train['transmission'].value_counts()

Manual       1948
Automatic    1660
Semi-Auto    1351
Other           1
Name: transmission, dtype: int64

In [9]:
X_train[X_train['transmission']=='Other']

,brand,year,transmission,mileage,fuelType,tax,mpg,engineSize
64,skoda,2019,Other,3500,1,145.0,47.1,1.0


In [10]:
X_test[X_test['transmission']=='Other']

,brand,year,transmission,mileage,fuelType,tax,mpg,engineSize
2386,merc,2015,Other,39000,0,150.0,56.5,2.1


In [11]:
X_train.at[(X_train['transmission']=='Other').index,'transmission']='Semi-Auto'
X_test.at[(X_test['transmission']=='Other').index,'transmission']='Semi-Auto'

In [12]:
X_train['transmission']=X_train['transmission'].map({"Manual":0,"Automatic":1,"Semi-Auto":2})
X_test['transmission']=X_test['transmission'].map({"Manual":0,"Automatic":1,"Semi-Auto":2})

In [13]:
X_train.describe()

,year,transmission,mileage,fuelType,tax,mpg,engineSize
count,4960.000000,4960.0,4960.000000,4960.000000,4960.000000,4960.000000,4960.000000
mean,2016.737903,2.0,24956.286895,0.447177,152.332661,50.370766,2.124375
std,2.884035,0.0,24443.333662,0.599819,82.403844,35.746505,0.789368
min,1997.000000,2.0,1.000000,0.000000,0.000000,2.800000,0.000000
25%,2016.000000,2.0,5641.250000,0.000000,145.000000,38.700000,1.600000
50%,2017.000000,2.0,19000.000000,0.000000,145.000000,47.100000,2.000000
75%,2019.000000,2.0,36702.000000,1.000000,150.000000,54.300000,2.800000
max,2020.000000,2.0,259000.000000,4.000000,580.000000,470.800000,6.600000


In [14]:
X_train=X_train.values
X_test=X_test.values

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train[:,2]=sc.fit_transform(X_train[:,2].reshape(-1,1)).flatten()
X_test[:,2]=sc.transform(X_test[:,2].reshape(-1,1)).flatten()

In [16]:
X_train.shape

(4960, 8)

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
X_train=np.array(ct.fit_transform(X_train))
X_test=np.array(ct.transform(X_test))

In [18]:
X_train

array([[0.0, 0.0, 0.0, ..., 145.0, 39.8, 2.2],
       [0.0, 0.0, 0.0, ..., 125.0, 60.1, 2.0],
       [1.0, 0.0, 0.0, ..., 145.0, 29.1, 2.9],
       ...,
       [0.0, 0.0, 0.0, ..., 30.0, 60.1, 2.0],
       [1.0, 0.0, 0.0, ..., 145.0, 62.8, 3.0],
       [0.0, 0.0, 0.0, ..., 150.0, 56.5, 3.0]], dtype=object)

In [19]:
y_train.drop(columns='carID',axis=1,inplace=True)
y_train=y_train.values

In [20]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [21]:
y_pred=lr.predict(X_test)

In [22]:
y_pred

array([[ 38176.6022423 ],
       [ 28043.8010852 ],
       [ 42292.8140781 ],
       ...,
       [ 19700.48736298],
       [ 26571.71247631],
       [-16806.97282386]])

In [23]:
y_test

array([[38000],
       [23495],
       [59999],
       ...,
       [21995],
       [18700],
       [ 2999]], dtype=int64)

In [24]:
from sklearn.metrics import mean_squared_error,r2_score
print("MSE = ",mean_squared_error(y_test,y_pred))
print("R2 Score = ",r2_score(y_test,y_pred))

MSE =  83956588.08995914
R2 Score =  0.6952367259870611


In [25]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor()
regressor.fit(X_train,y_train)

C:\Users\dell\AppData\Local\Temp/ipykernel_24928/1077422990.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,y_train)


RandomForestRegressor()

In [26]:
rf_ypred=regressor.predict(X_test)
rf_ypred=rf_ypred.reshape(-1,1)
rf_ypred

array([[38510.55],
       [23735.44],
       [57376.31],
       ...,
       [20864.88],
       [22809.95],
       [ 3158.59]])

In [27]:
y_test

array([[38000],
       [23495],
       [59999],
       ...,
       [21995],
       [18700],
       [ 2999]], dtype=int64)

In [28]:
print(mean_squared_error(y_test,rf_ypred))
print(r2_score(y_test,rf_ypred))

14646394.332251534
0.94683343867668


In [29]:
import joblib
joblib.dump(regressor,'model.joblib')

['model.joblib']